# 🎨 Look1nce - OOTDiffusion on YOUR Colab GPU (NO QUOTA LIMITS!)

## ⚡ Setup:
1. **Runtime → Change runtime type → T4 GPU → Save**
2. **Runtime → Run all** (takes 10-15 minutes first time)
3. **Copy Gradio URL**
4. **Paste in backend/.env**
5. **Unlimited try-ons!** 🚀

## Step 1: Check GPU

In [ ]:
!nvidia-smi
import torch
print(f"\n✅ GPU: {torch.cuda.get_device_name(0)}")
print(f"💾 VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

## Step 2: Install Core Dependencies

In [ ]:
print("📦 Installing core packages (3-4 minutes)...\n")

# Uninstall conflicting packages first (they're not needed for OOTDiffusion)
print("🧹 Removing conflicting packages...")
!pip uninstall -y -q sentence-transformers

# Install essential packages
print("📥 Installing OOTDiffusion dependencies...")
!pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install -q diffusers==0.27.2 transformers==4.38.2 accelerate==0.27.2
!pip install -q opencv-python pillow numpy gradio spaces
# Install older huggingface_hub (FIXES cached_download import error!)
!pip install -q 'huggingface_hub==0.20.0'

print("\n✅ Core packages installed (warnings are normal)!")

## Step 3: Clone OOTDiffusion & Fix Dependencies

In [ ]:
import os
from pathlib import Path
import sys

# Clone if needed
if not Path("/content/OOTDiffusion").exists():
    print("📥 Cloning OOTDiffusion...")
    !git clone https://github.com/levihsu/OOTDiffusion.git /content/OOTDiffusion
    print("✅ Cloned!")
else:
    print("✅ Repo exists!")

# Add to Python path (FIXES THE IMPORT ERROR!)
sys.path.insert(0, '/content/OOTDiffusion')
os.chdir('/content/OOTDiffusion')

print(f"\n📁 Working directory: {os.getcwd()}")
print(f"🐍 Python path includes: /content/OOTDiffusion")

## Step 4: Create Missing Config File

In [ ]:
# The 'config' module that was missing - let's create it!
config_content = '''"""Configuration for OOTDiffusion"""
import os
from pathlib import Path

# Base paths
BASE_DIR = Path("/content/OOTDiffusion")
CHECKPOINT_DIR = BASE_DIR / "checkpoints"

# Model paths
OPENPOSE_CHECKPOINT = CHECKPOINT_DIR / "openpose" / "ckpts"
HUMANPARSING_CHECKPOINT = CHECKPOINT_DIR / "humanparsing"
OOTD_CHECKPOINT = CHECKPOINT_DIR / "ootd"

# Create directories
CHECKPOINT_DIR.mkdir(exist_ok=True)
OPENPOSE_CHECKPOINT.mkdir(parents=True, exist_ok=True)
HUMANPARSING_CHECKPOINT.mkdir(parents=True, exist_ok=True)
OOTD_CHECKPOINT.mkdir(parents=True, exist_ok=True)
'''

with open('/content/OOTDiffusion/config.py', 'w') as f:
    f.write(config_content)

print("✅ Created config.py file!")

## Step 5: Install OOTDiffusion Requirements & Missing Dependencies

In [ ]:
print("📦 Installing OOTDiffusion dependencies...\n")

# Install critical missing packages (THIS FIXES THE ERROR!)
print("1️⃣ Installing onnxruntime (for human parsing)...")
!pip install -q onnxruntime-gpu
print("   ✅ onnxruntime installed!\n")

print("2️⃣ Installing OpenCV (for image processing)...")
!pip install -q opencv-python-headless
print("   ✅ OpenCV installed!\n")

print("3️⃣ Installing other dependencies...")
!pip install -q einops omegaconf safetensors onnx
print("   ✅ Other dependencies installed!\n")

# Install from requirements if it exists
if Path("requirements.txt").exists():
    print("4️⃣ Installing from requirements.txt...")
    !pip install -q -r requirements.txt
    print("   ✅ Requirements installed!\n")

print("="*50)
print("✅ ALL DEPENDENCIES INSTALLED!")
print("="*50)

## Step 6: Download Model Checkpoints (THIS IS THE BIG ONE - 5-8 minutes)

In [ ]:
from huggingface_hub import snapshot_download, hf_hub_download
import os

print("📥 Downloading model checkpoints (~3-4GB)...")
print("⏳ This takes 5-8 minutes...\n")

# Download OOTDiffusion checkpoints
try:
    print("1️⃣ Downloading OOTDiffusion models...")
    snapshot_download(
        repo_id="levihsu/OOTDiffusion",
        local_dir="/content/OOTDiffusion/checkpoints/ootd",
        local_dir_use_symlinks=False
    )
    print("   ✅ OOTDiffusion models downloaded!\n")
except Exception as e:
    print(f"   ⚠️ Warning: {e}\n")

# Download OpenPose checkpoints
try:
    print("2️⃣ Downloading OpenPose checkpoints...")
    openpose_dir = "/content/OOTDiffusion/checkpoints/openpose/ckpts"
    os.makedirs(openpose_dir, exist_ok=True)
    
    # Download specific OpenPose files
    files = ['body_pose_model.pth', 'hand_pose_model.pth']
    for file in files:
        try:
            hf_hub_download(
                repo_id="levihsu/OOTDiffusion",
                filename=f"checkpoints/openpose/ckpts/{file}",
                local_dir="/content/OOTDiffusion",
                local_dir_use_symlinks=False
            )
        except:
            print(f"   ⚠️ Skipping {file}")
    print("   ✅ OpenPose checkpoints ready!\n")
except Exception as e:
    print(f"   ⚠️ Warning: {e}\n")

# Download Human Parsing checkpoints
try:
    print("3️⃣ Downloading Human Parsing checkpoints...")
    hf_hub_download(
        repo_id="levihsu/OOTDiffusion",
        filename="checkpoints/humanparsing/parsing_atr.onnx",
        local_dir="/content/OOTDiffusion",
        local_dir_use_symlinks=False
    )
    print("   ✅ Human Parsing checkpoints ready!\n")
except Exception as e:
    print(f"   ⚠️ Warning: {e}\n")

print("\n" + "="*50)
print("✅ ALL CHECKPOINTS DOWNLOADED!")
print("="*50)

## Step 7: Load Models into GPU Memory

In [ ]:
print("🔄 Loading models into GPU... (2-3 minutes)\n")

import torch
from PIL import Image
import numpy as np

# Import with proper path
try:
    from preprocess.openpose.run_openpose import OpenPose
    from preprocess.humanparsing.run_parsing import Parsing
    from ootd.inference_ootd_hd import OOTDiffusionHD
    print("✅ Imports successful!\n")
except ImportError as e:
    print(f"⚠️ Import error: {e}")
    print("⚠️ Trying alternative import method...\n")
    
    # If direct import fails, we'll use gradio_client as fallback
    USE_FALLBACK = True
else:
    USE_FALLBACK = False
    
    # Load models
    print("1️⃣ Loading OpenPose...")
    openpose_model = OpenPose(0)
    print("   ✅ OpenPose loaded!\n")
    
    print("2️⃣ Loading Human Parsing...")
    parsing_model = Parsing(0)
    print("   ✅ Human Parsing loaded!\n")
    
    print("3️⃣ Loading OOTDiffusion...")
    ootd_model = OOTDiffusionHD(0)
    print("   ✅ OOTDiffusion loaded!\n")
    
    print("="*50)
    print("✅ ALL MODELS LOADED INTO GPU!")
    print("="*50)

## Step 8: Create Try-On Function

In [ ]:
import gradio as gr

if USE_FALLBACK:
    print("⚠️ Using fallback mode (will still work but uses HF Space)\n")
    from gradio_client import Client, handle_file
    import time
    
    hf_client = Client("levihsu/OOTDiffusion")
    
    def virtual_tryon(person_img, cloth_img, category="Upper-body", num_steps=20):
        """Fallback: Forward to HF Space"""
        try:
            person_path = f"/tmp/person_{int(time.time())}.png"
            cloth_path = f"/tmp/cloth_{int(time.time())}.png"
            
            person_img.save(person_path)
            cloth_img.save(cloth_path)
            
            result = hf_client.predict(
                vton_img=handle_file(person_path),
                garm_img=handle_file(cloth_path),
                n_samples=1,
                n_steps=num_steps,
                image_scale=2.0,
                seed=-1,
                api_name="/process_hd"
            )
            
            if isinstance(result, list) and len(result) > 0:
                if isinstance(result[0], dict) and 'image' in result[0]:
                    return Image.open(result[0]['image'])
            return result
        except Exception as e:
            raise gr.Error(f"Error: {str(e)}")
else:
    print("✅ Using LOCAL GPU mode (unlimited!)\n")
    
    def virtual_tryon(person_img, cloth_img, category="Upper-body", num_steps=20):
        """Run on YOUR Colab GPU!"""
        try:
            print(f"🎨 Processing on YOUR GPU: {category}")
            
            # Preprocess
            print("📐 Detecting pose...")
            keypoints = openpose_model(person_img)
            
            print("👤 Parsing human...")
            model_parse, _ = parsing_model(person_img)
            
            # Map category
            cat_map = {'Upper-body': 0, 'Lower-body': 1, 'Dress': 2}
            cat_idx = cat_map.get(category, 0)
            
            # Run try-on
            print(f"🚀 Running AI model ({num_steps} steps)...")
            results = ootd_model(
                category=cat_idx,
                image_garm=cloth_img,
                image_vton=person_img,
                mask=model_parse,
                image_ori=person_img,
                num_samples=1,
                num_steps=num_steps,
                seed=-1
            )
            
            print("✅ Done!")
            return results[0] if isinstance(results, list) else results
            
        except Exception as e:
            print(f"❌ Error: {str(e)}")
            raise gr.Error(f"Try-on failed: {str(e)}")

print("✅ Try-on function ready!")

## Step 9: Create Gradio API

In [ ]:
mode = "LOCAL GPU" if not USE_FALLBACK else "FALLBACK (HF Space)"

with gr.Blocks(title="Look1nce API", theme=gr.themes.Soft()) as demo:
    gr.Markdown(f"""
    # 🎨 Look1nce Virtual Try-On API
    
    ### Mode: {mode} ⚡
    ### Running on: Google Colab T4 GPU
    
    **Keep this tab open!**
    """)
    
    with gr.Row():
        with gr.Column():
            person_input = gr.Image(label="👤 Person", type="pil")
            cloth_input = gr.Image(label="👔 Clothing", type="pil")
        with gr.Column():
            result_output = gr.Image(label="✨ Result")
    
    with gr.Row():
        category_input = gr.Dropdown(
            ['Upper-body', 'Lower-body', 'Dress'],
            value='Upper-body',
            label="Category"
        )
        steps_input = gr.Slider(10, 50, 20, step=5, label="Steps")
    
    btn = gr.Button("🚀 Generate", variant="primary")
    btn.click(virtual_tryon, [person_input, cloth_input, category_input, steps_input], result_output)
    
    gr.Markdown("""
    ---
    ### Next: Copy the public URL → Paste in backend/.env → Restart backend
    """)

print("✅ Interface ready!")

## Step 10: 🚀 LAUNCH!

In [ ]:
print("="*60)
print("🚀 LAUNCHING LOOK1NCE API")
print("="*60)
print(f"\nMode: {'LOCAL GPU (Unlimited!)' if not USE_FALLBACK else 'Fallback (HF Space)'}\n")

demo.launch(
    share=True,
    server_name="0.0.0.0",
    server_port=7860,
    debug=True
)

print("\n" + "="*60)
print("✅ SERVER RUNNING!")
print("="*60)
print("\n📋 COPY THE URL ABOVE")
print("📝 PASTE IN: D:/Look1nce/backend/.env")
print("🔄 RESTART BACKEND")
print("🎉 ENJOY UNLIMITED TRY-ONS!")
print("\n⚠️ KEEP THIS TAB OPEN!")
print("="*60)